In [ ]:
import pandas as pd
import overpy

In [ ]:
# Define the bounding box
min_lon = 9.6844408
min_lat = 52.0779586
max_lon = 9.9935122
max_lat = 52.3471095
bbox = f"({min_lat}, {min_lon}, {max_lat}, {max_lon})"

https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_API_by_Example
https://wiki.openstreetmap.org/wiki/Key:amenity#Sustenance

In [ ]:
nodes = {
    'shop': ['supermarket', 'doityourself', 'hardware',
             'department_store', 'mall', 'wholesale'],
    'amenity': ['school', 'kindergarten', 'college',
                'university', 'clinic', 'hospital'],
    'leisure': ['swimming_pool', 'fitness_centre',
                'sports_centre'],
    'public_transport': ['stop_position', 'station',
                         'plattform']
}

tag_to_column = {
    'name': 'name',
    'addr:city': 'city',
    'addr:suburb': 'suburb',
    'addr:postcode': 'postcode',
    'addr:street': 'street',
    'addr:housenumber': 'housenumber',
    'operator': 'operator',
    'local_ref': 'local_ref',
    'bus': 'bus',
    'tram': 'tram',
    'network': 'network',
    'railway': 'railway',
    'brand': 'brand',
}

In [ ]:
# Initialize the Overpass API object
api = overpy.Overpass()

# Initialize an empty list to hold dictionaries of data
data_list = []

# Iterate through the nodes and their tags to make API queries
for key, tags in nodes.items():
    for tag in tags:
        query = f"""
            [out:json];
            nw[{key}={tag}]{bbox};
            out center;
            """
        result = api.query(query)

        for node in result.nodes + result.ways:
            # Create a dictionary with the common columns
            node_dict = {
                'type': node._type_value,
                'category': key,
                'subcategory': tag,
                'id': node.id,
            }

            # Store coordinates of nodes
            if node_dict['type'] == 'node':
                node_dict['lat'] = node.lat
                node_dict['lon'] = node.lon
            elif node_dict['type'] != 'node':
                node_dict['lat'] = node.center_lat
                node_dict['lon'] = node.center_lon
                node_dict['node_ids'] = node._node_ids

            # Add tags as new keys in the dictionary
            for tag_key, tag_value in node.tags.items():
                if tag_key in tag_to_column:
                    node_dict[tag_to_column[tag_key]] = tag_value

            # Append the dictionary to the data list
            data_list.append(node_dict)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Save the DataFrame as a CSV file
df.to_csv('osm_data.csv', index=False)

### confirm that the main attributes aren't null

In [ ]:
df.isnull().any()